## GrEx2: XYZ Company Data for Marketing and CRM ##

## Part I ##

<ol>
  <h3>Steps.</h3>
  In order to get the data we need for GrEx2, we weed to complete the following steps.
  <li>If we are not on campus, we need to first connect to the NU VPN.</li>
  <li>Next we will need to use a SSH client (e.g. PuTTy) to connect to the SPN Elasticsearch server. </li>
  <li>Once our login information is authenticated, we will the psql client to connect to the xyz database on the PostgreSQL server.</li>
  <li>Within the pilot schema of the xzy database, we will execute three \copy commands to export the customer, item and mail tables into three separate CSV files. These files will get placed in our SPS Elasticsearch server current working directory.</li>
  <li>Next we will use an SFTP client (e.g. CyberDuck) to connect to the SPS Elasticsearch server. Once connected, we should see our current working directory and any folder/files that are on it. </li>
  <li>Download the customer, item and mail CSV files from our current working directory to our local machine. </li>
  <li>Create a pandas DataFrame for each of the CSV files we downloaded.</li>
</ol>

In [1]:
import pandas as pd #contains I/O functions for various file types include (csv and excel files) 
import numpy as np  
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Read in the CSV files using pandas to create the necessary DataFrames 
item_df = pd.read_csv('.\\item.csv', sep = ',')
mail_df = pd.read_csv('.\\mail.csv', sep = ',')
customer_df = pd.read_csv('.\\customer.csv', sep = ',')

In [3]:
# Verify the content of item_df DataFrame by displaying column names
item_df.columns

Index(['acctno', 'qty', 'trandate', 'tran_channel', 'price', 'totamt',
       'orderno', 'deptdescr'],
      dtype='object')

In [4]:
# Verify the content of mail_df DataFrame by displaying column names
mail_df.columns

Index(['acctno', 'mail_1', 'mail_2', 'mail_3', 'mail_4', 'mail_5', 'mail_6',
       'mail_7', 'mail_8', 'mail_9', 'mail_10', 'mail_11', 'mail_12',
       'mail_13', 'mail_14', 'mail_15', 'mail_16'],
      dtype='object')

In [5]:
# Verify the content of customer_df DataFrame by displaying column names
customer_df.columns

Index(['acctno', 'zip', 'zip4', 'ltd_sales', 'ltd_transactions',
       'ytd_sales_2009', 'ytd_transactions_2009', 'channel_acquisition',
       'buyer_status', 'zip9_supercode',
       ...
       'zprchonp', 'ztennis', 'ztennisp', 'ztravdom', 'ztravdop', 'ztravfor',
       'ztravfop', 'zvoluntr', 'zvoluntp', 'endofline'],
      dtype='object', length=451)

In [6]:
# Verify the data type of the columns of one (item_df) of our DataFrames
item_df.dtypes
# item_df has three different data types (object, integer, float) which vary across columns  

acctno           object
qty               int64
trandate         object
tran_channel     object
price           float64
totamt          float64
orderno          object
deptdescr        object
dtype: object

In [7]:
# Verify the content of the first 4 rows of one of our DataFrames
# Let's use item_df
item_df.head(4)

,acctno,qty,trandate,tran_channel,price,totamt,orderno,deptdescr
0,AAAAPSSYY,1,12/19/2009,RT,89.85,89.85,CCXUNIKXXXVI,Portable Electronics
1,AAAAPSSYY,1,12/19/2009,RT,89.85,89.85,CCXUNIKXXXVI,Portable Electronics
2,AAAASDQWP,1,11/15/2009,RT,79.50,79.50,CCXNEIVXXXVI,Mobile Electronic Accessories
3,AAAASDQWP,2,12/18/2009,RT,79.50,159.00,CCXRCREXXXKI,Mobile Electronic Accessories


## Part 2 ##

<h3> Steps: </h3>
      -Write each of the newly created DataFrames to a local SQLite DB named xyz.db. Include only data for active buyers in these tables. Verify that the tables have been written to the SQLite DB correctly. 

In [8]:
# Create the active_customer DataFrame
# An active customer is considered a customer who has "ACTIVE" listed in their respective 'buyer_status' column 
active_customer = customer_df[customer_df['buyer_status']=='ACTIVE']

In [9]:
# Create the active_item DataFrame
# Use the 'acctno' column from the active_customer DataFrame in order to filter the active customers from item_df
pre_active_item = item_df.merge(active_customer, how='inner', on=['acctno'])

In [10]:
active_item = pre_active_item[['acctno', 'qty','trandate','tran_channel', 'price', 'totamt', 'orderno', 'deptdescr']]

In [11]:
# Create the active_mail DataFrame
# Use the 'acctno' column from the active_customer DataFrame in order to filter the active customers from mail_df
pre_active_mail = mail_df.merge(active_customer, how='inner', on=['acctno'])

In [12]:
active_mail = pre_active_mail[['acctno','mail_1','mail_2','mail_3','mail_4','mail_5','mail_6','mail_7','mail_8','mail_9','mail_10','mail_11','mail_12','mail_13','mail_14','mail_15','mail_16']]

In [13]:
# Import library and packages used to create xyz.db 
import sqlalchemy 
from sqlalchemy import create_engine

In [14]:
# Create SQLAlchemy engine to connect to xyz.db
# The file, xyz.db, will be created and saved to the current directory
engine=create_engine('sqlite:///xyz.db')  
conn=engine.connect()

In [15]:
# Write active_customer, active_item and active_mail to a local SQLite DB named xyz.db
active_customer.to_sql('active_customer',conn,index=False, if_exists='replace')
active_item.to_sql('active_item',conn,index=False, if_exists='replace')
active_mail.to_sql('active_mail',conn,index=False, if_exists='replace')

In [16]:
# Verify that the tables have been written to our SQLite DB correctly
table_names_list = engine.table_names()
print(*table_names_list, sep=', ')

active_customer, active_item, active_mail


In [17]:
# We will also verify the tables have been written to our SQLIte DB correctly by returning the rows of each table by querying
# We are going to print the results from our query and compare with resuls from using shape on the DataFrame
active_customer_count = pd.read_sql_query("SELECT COUNT(*) FROM active_customer",conn)
print (active_customer_count)
active_customer.shape

   COUNT(*)
0     17491


(17491, 451)

In [18]:
# Verifying active_item
active_item_count = pd.read_sql_query("SELECT COUNT(*) FROM active_item",conn)
print (active_item_count)
active_item.shape

   COUNT(*)
0     77121


(77121, 8)

In [19]:
# Verifying active_mail
active_mail_count = pd.read_sql_query("SELECT COUNT(*) FROM active_mail",conn)
print (active_mail_count)
active_mail.shape

   COUNT(*)
0     13714


(13714, 17)

## Part 3 ##

<h3> Steps: </h3>
      -Using the same data we used for Part 1, create a new DataFrame called custSum that we will also write as a table to xyz.db. This table should have one row per customer record.

In [20]:
# Add a heavy_buyer column to the customer DataFrame with values of either "Y" or "N" to indicate if the customer is a heavy buyer
# A heavy buyer is a customer whose LTD purchasing in 2009 (ltd_sales) is greater than 90% of the LTD purchasing of all customers
# We need to determine the 90th percentile of the ltd_sales column entries using quantile method
ltd_sale_90th = customer_df['ltd_sales'].quantile(q=0.9)
print (ltd_sale_90th)

2145.0


In [21]:
# Create a heavy_buyer condition using np.where and our previously created ltd_sales_90th variable
customer_df['heavy_buyer'] = np.where((customer_df.ltd_sales>ltd_sale_90th), "Y", "N")

In [22]:
# Check customer_df to see if new column and condition was applied correctly
# We can see that acctno GGGDYHHLL had ltd_sales greater than 2145 and was marked as "Y" under heavy_buyer column
customer_df[['acctno', 'ltd_sales', 'heavy_buyer']].head(15)

,acctno,ltd_sales,heavy_buyer
0,HQYHLQHL,420.0,N
1,LWYGGQPLW,354.0,N
2,GAPSQASLA,18.0,N
3,GAGQDSHLW,174.0,N
4,GGSSQWLDG,282.0,N
5,AAQPPQQA,810.0,N
6,LPPQWGSLL,60.0,N
7,SPDGWHWPA,30.0,N
8,PWAYSDADA,1806.0,N
9,LDYWPQHYY,120.0,N


In [23]:
# We will verify our heavy_buyer coding by using a crosstab that shows how many heavy_buyers are in the customer_df
# For consistency, we are going to use an indicator that shows greater than 90th percentile as opposed to at least, which is 
# what directions states.  
# We can see that there are 4997 heavy_buyer entries (i.e customers)
heavy_buyer_crtab = pd.crosstab(np.where(customer_df.heavy_buyer=="N", "ltd_sales<90th%?","ltd_sales>90th%?"), customer_df.heavy_buyer)
heavy_buyer_crtab.index.rename("ltd sales check", inplace=True)
heavy_buyer_crtab

heavy_buyer,N,Y
ltd sales check,,
ltd_sales<90th%?,45003,0
ltd_sales>90th%?,0,4997


In [24]:
# Just another way to verify we have 4997 heavy_buyer entries
customer_df['heavy_buyer'].value_counts(dropna=False)

N    45003
Y     4997
Name: heavy_buyer, dtype: int64

In [25]:
# We need to now add columns to customer_df that indicate whether the customer has the following credit cards:
# AMEX, Discover, VISA and Mastercard
# A customer will get a "Y" in new columns we are creating if they have "Y" in either amex_reg or amex_prem record
# We will use np.where and | to form compound conditionals 
# Create AMEX column 
customer_df['has_amex'] = np.where((customer_df.amex_reg=='Y') | (customer_df.amex_prem =='Y'), "Y", "N")

In [26]:
# Create Discover column 
customer_df['has_disc'] = np.where((customer_df.disc_reg=='Y') | (customer_df.disc_prem=='Y'), "Y", "N")

In [27]:
# Create VISA column
customer_df['has_visa'] = np.where((customer_df.visa_reg == 'Y') | (customer_df.visa_prem=='Y'), "Y", "N")

In [28]:
# Create Mastercard column
customer_df['has_mc'] = np.where((customer_df.mc_reg == 'Y') | (customer_df.mc_prem=='Y'), "Y", "N")

In [29]:
# Check VISA data 
customer_df[['visa_reg', 'visa_prem','has_visa']].head(10)
# We can see that our code is correct. Note that NaN values were given a "N" under has_visa which makes sense 

,visa_reg,visa_prem,has_visa
0,Y,U,Y
1,NaN,NaN,N
2,NaN,NaN,N
3,NaN,NaN,N
4,NaN,NaN,N
5,U,U,N
6,NaN,NaN,N
7,NaN,NaN,N
8,NaN,NaN,N
9,NaN,NaN,N


In [30]:
# We will use crosstab again to verify our code is correct using VISA data
visa_crtab = pd.crosstab([customer_df.visa_reg, customer_df.visa_prem], customer_df.has_visa)
visa_crtab
# We can see that there are 23501 customers that have either a visa_reg or visa_prem, which means they have at least one type

has_visa               N      Y
visa_reg visa_prem             
U        U          8104      0
         Y             0     86
Y        U             0  23045
         Y             0    370

In [31]:
# We can now begin constructing our custSum DataFrame
# We will add acctno so we can relate custSum to other tables in our xyz database
# The other columns were added based on instructions 
custSum = customer_df[['acctno', 'zip', 'zip4', 'ltd_sales', 'amex_reg', 'amex_prem',
       'med_inc', 'adult1_g', 'adult1_g', 'disc_reg', 'disc_prem', 'visa_reg',
       'visa_prem', 'mc_reg', 'mc_prem', 'heavy_buyer', 'has_amex', 'has_disc',
       'has_mc', 'has_visa']]

In [32]:
# Write custSum to our xyz database 
custSum.to_sql('custSum',conn,index=False, if_exists='replace')

In [33]:
# Verify that the tables have been written to our SQLite DB correctly
table_names_list = engine.table_names()
print(*table_names_list, sep=', ')

active_customer, active_item, active_mail, custSum


In [34]:
# We will also verify the custSum table have been written to our SQLIte DB correctly by returning the rows by querying
# We are going to print the results from our query and compare with resuls from using shape on the DataFrame
custSum_count = pd.read_sql_query("SELECT COUNT(*) FROM custSum",conn)
print (custSum_count)
custSum.shape

   COUNT(*)
0     50000


(50000, 20)

## Part 4 ##

<h3> Steps: </h3>
      -Create a new pandas DataFrame called market_df that will be used for target maketing and write it out to a headered csv ﬁle.

In [35]:
# We are going to create a DataFrame called "market_df" which will list both lapsed and active customers and show us the total amount of products
# they purchased in seven distinct product categories
# First, we will create a DataFrame "pre_market_df" which include the acctno, buyer_status and ltd_sales columns from customer_df
# This DataFrame will have rows for only active or lapsed customers
pre_market_df = customer_df[customer_df['buyer_status']!= "INACTIVE"]
pre_market_df = pre_market_df[['acctno', 'buyer_status', 'ltd_sales']]

In [36]:
# We need to use a copy of the item_df in order to get the data for how many products were purchased by category
# We will use the groupby() method on acctno in order to group all the customers into one row since some customers made multiple purchases
# Using the unstack() method on the deptdescr column shows how many products were purchased in each category 
updated_item_df = item_df.copy()
pre_market2_df=updated_item_df.groupby('acctno')['deptdescr'].value_counts().unstack().fillna(0).reset_index()
del pre_market2_df.columns.name
pre_market2_df.head(10)

,acctno,Appliances,Cameras & Camcorder Accessori,Home Audio,Mobile Electronic Accessories,Mobile Electronics,Portable Electronics,Small Appliances
0,AAAAPSSYY,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,AAAASDQWP,0.0,0.0,0.0,4.0,0.0,0.0,1.0
2,AAAASYHQW,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,AAAASYLYG,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,AAAASYPHD,0.0,0.0,0.0,4.0,0.0,0.0,0.0
5,AAAASYPPG,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,AAAASYPYH,0.0,0.0,1.0,2.0,0.0,0.0,0.0
7,AAAASYYPA,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,AAADPSLAP,0.0,0.0,0.0,3.0,0.0,0.0,0.0
9,AAAGPLGDD,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [37]:
# Now we can create the'market_df' by merging the 'pre_market_df' and 'pre_market2_df' 
# Essentially this line of code selects all the active and lapsed customers which is what we want
market_df = pre_market_df.merge(pre_market2_df, on='acctno', how='outer')

In [38]:
# The following lines of code are being exeucted in order to turn all NaN values into 0
# 0 indicaates that the customer did not make a purchase in that category 
market_df['Appliances'].replace(np.nan,0, inplace=True)
market_df['Cameras & Camcorder Accessori'].replace(np.nan,0, inplace=True)
market_df['Home Audio'].replace(np.nan,0, inplace=True)
market_df['Mobile Electronic Accessories'].replace(np.nan,0, inplace=True)
market_df['Mobile Electronics'].replace(np.nan,0, inplace=True)
market_df['Portable Electronics'].replace(np.nan,0, inplace=True)
market_df['Small Appliances'].replace(np.nan,0, inplace=True)

In [39]:
# We will use the sort_values function to display our final DataFrame 
market_df.sort_values(by='acctno').head(5)

,acctno,buyer_status,ltd_sales,Appliances,Cameras & Camcorder Accessori,Home Audio,Mobile Electronic Accessories,Mobile Electronics,Portable Electronics,Small Appliances
27747,AAAAPAHGQ,LAPSED,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5580,AAAAPSSYY,ACTIVE,2160.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
2198,AAAASDQPY,LAPSED,960.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2653,AAAASDQWP,ACTIVE,3513.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0
20642,AAAASYHQW,ACTIVE,858.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [40]:
# We will write market_df as a headered CSV file
market_df.to_csv('market_df.csv', index=False)

In [41]:
# Check to see that the CSV file was written correctly by reading it back in and use the equals method
# If the output is True then we can confirm that the market_df DataFrame was written correctly to the CSV file
verify_market_df = pd.read_csv('.\\market_df.csv', sep=',' )
verify_market_df.equals(market_df)


True

In [42]:
# We will also store the 'market_df' DataFrame in a shelve database called 'xyzShelf'
# Import the shelve module and create empty 'xyzShelf'
import shelve 
xyz_db=shelve.open('xyzShelf')  

In [43]:
# Along with storing the 'market_df' DataFrame, we will also go ahead and store 'custSum', 'active_customer', 'active_item', and 'active_mail'
# We will assign each DataFrame to a key in the shelf and then close the shelf
xyz_db['market_df'] = market_df
xyz_db['custSum'] = custSum
xyz_db['active_customer'] = active_customer
xyz_db['active_iteml'] = active_item
xyz_db['active_mail'] = active_mail
xyz_db.close() 

In [44]:
# Check to see that our DataFrames were stored correctly in the 'xyzShelf' by opening it and then reading in two keys
# We will use the equals method to verify everything is correct
# If the output is True then we can confirm that the DataFrames were written correctly to the 'xyzShelf'
# We will check 'custSum'
xyzShelfBack = shelve.open('xyzShelf')
verify_custSum = xyzShelfBack['custSum']
verify_custSum.equals(custSum)

True

In [45]:
# Let's also check 'active_mail'
verify_active_mail = xyzShelfBack['active_mail']
verify_active_mail.equals(active_mail)

True